In [17]:
log_root = r'D:\issp_data\features_total'
out_folder = r'C:\Users\howard810804\Desktop\proj_issp'
fraction = 1

model_folder = r'D:\issp_data\model'
test_size = 0.25

In [18]:
import os
import warnings
warnings.filterwarnings('ignore')

from typing import Tuple, List

import dask.dataframe as dd
import gzip
import numpy as np
import pickle
import time

from distributed.protocol.serialize import register_generic
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from tpot import TPOTClassifier
register_generic(TPOTClassifier)

import xgboost as xgb

In [19]:
def load_data(st: int, ed: int) -> Tuple['np.array']:
    path = os.path.join(log_root, sorted(os.listdir(log_root))[0])
    col_nums = dd.read_csv(path, compression='zip').shape[-1]
    arr_X, arr_y = np.zeros(col_nums).reshape((-1, col_nums)), np.array([0])
    
    for fl in os.listdir(log_root)[(st - 1) * 3: ed * 3]:
        cata = int(fl[31])
        path = os.path.join(log_root, fl)
        print(path)
        tmp = dd.read_csv(path, compression='zip')\
                .compute()\
                .values
        
        arr_X = np.vstack((arr_X, tmp))
        arr_y = np.hstack((arr_y, np.array([cata] * tmp.shape[0])))
        
    return arr_X[1: , :], arr_y[1: ]

In [ ]:
# def load_data_month(fl_ls: List[str], fraction: int) -> Tuple['np.array']:
#     _ = os.path.join(log_root, sorted(os.listdir(log_root))[0])
#     arr_X, arr_y = np.array([])\
#             .reshape((-1, dd.read_csv(_, compression='zip')\
#             .shape[-1])), np.array([])
    
#     for fl in fl_ls:
#         cata, path = int(fl[31]), os.path.join(log_root, fl)
#         tmp = dd.read_csv(path, compression='zip')\
#                 .compute()\
#                 .values
        
#         arr_X = np.vstack((arr_X, tmp))
#         arr_y = np.hstack((arr_y, np.array([cata] * tmp.shape[0])))
        
#     X, y = resample(arr_X, arr_y, n_samples=arr_X.shape[0]//fraction,
#                     replace=False, stratify=arr_y, random_state=None)
    
#     X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=test_size, random_state=None)
    
#     return X_train, X_test, y_train, y_test

# # fl_ls = os.listdir(log_root)
# # print(fl_ls)
# # load_data_month(fl_ls[:3], 30)

In [20]:
def feature_importances(model_obj) -> List:
    path = lambda x: os.path.join(x, os.listdir(x)[0])
    df = dd.read_csv(path(log_root), sep='\t', compression='zip')
    cols = list(df.compute())[0].split(',')

    features = model_obj.feature_importances_\
                    .tolist()
    result = [(fea, col) for (fea, col) in zip(features, cols)]
    return sorted(result, reverse=True)

In [22]:
arr_X, arr_y = load_data(1, 12)

D:\issp_data\features_total\s35809_MobileApp_201701_104628_0_features.csv
D:\issp_data\features_total\s35809_MobileApp_201701_104628_1_features.csv
D:\issp_data\features_total\s35809_MobileApp_201701_104628_2_features.csv
D:\issp_data\features_total\s35809_MobileApp_201702_104645_0_features.csv
D:\issp_data\features_total\s35809_MobileApp_201702_104645_1_features.csv
D:\issp_data\features_total\s35809_MobileApp_201702_104645_2_features.csv
D:\issp_data\features_total\s35809_MobileApp_201703_104646_0_features.csv
D:\issp_data\features_total\s35809_MobileApp_201703_104646_1_features.csv
D:\issp_data\features_total\s35809_MobileApp_201703_104646_2_features.csv
D:\issp_data\features_total\s35809_MobileApp_201704_104647_0_features.csv
D:\issp_data\features_total\s35809_MobileApp_201704_104647_1_features.csv
D:\issp_data\features_total\s35809_MobileApp_201704_104647_2_features.csv
D:\issp_data\features_total\s35809_MobileApp_201705_104648_0_features.csv
D:\issp_data\features_total\s35809_Mob

In [23]:
X, y = resample(arr_X, arr_y, n_samples=arr_X.shape[0]//fraction,
                replace=False, stratify=arr_y, random_state=None)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=None)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(19816, 65) (19816,)
(6606, 65) (6606,)


In [ ]:
config_dict = {
    'xgboost.XGBClassifier': {
    },
    'sklearn.ensemble.ExtraTreesClassifier': {
    },
    'sklearn.ensemble.GradientBoostingClassifier': {
    }
}

clf = TPOTClassifier(generations=3,
                     population_size=10, 
                     n_jobs=-1,
                     verbosity=2,
                     config_dict=config_dict,
                     random_state=None,
                     use_dask=True)

%time clf.fit(X_train, y_train)
print(clf.fitted_pipeline_.steps[1])

print(clf.score(X_test, y_test))

Generation 1 - Current best internal CV score: 0.7210337142279506


TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.


In [7]:
# # Random forest

# clf_rf = RandomForestClassifier(verbose=2, n_jobs=-1)
# %time clf_rf.fit(X_train, y_train)
# y_pred = clf_rf.predict(X_test)

# print('fraction:', fraction)
# print(clf_rf.score(X_test, y_test), '\n')
# print(confusion_matrix(y_test, y_pred), '\n')
# print(clf_rf.feature_importances_)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 10building tree 2 of 10building tree 3 of 10

building tree 4 of 10

building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  7.7min finished


Wall time: 7min 57s


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    9.0s finished


fraction: 1


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    8.4s finished


0.7777984675443165 

[[884004  97360  25730]
 [135350 395451  46059]
 [ 60000  75828 261874]] 

[0.02671566 0.01449006 0.00764694 0.00943158 0.01579771 0.01577381
 0.01572045 0.01582526 0.01825655 0.0208648  0.0173462  0.01741488
 0.02381033 0.05464337 0.02765648 0.02443092 0.05147059 0.10803755
 0.01943834 0.02124994 0.02140405 0.02553    0.03984851 0.02674785
 0.05684549 0.0293212  0.03324171 0.01733779 0.00979577 0.01111745
 0.00605091 0.00806537 0.00230594 0.00720775 0.00698933 0.00187512
 0.00652165 0.01025316 0.0100657  0.00681628 0.01028307 0.01240999
 0.01066363 0.00876685 0.00589019 0.00976296 0.0078401  0.00377352
 0.00175238 0.0064583  0.00445656 0.00781766 0.00519817 0.00214119
 0.00216748 0.00128221 0.00137513 0.00175563 0.00152245 0.0016021
 0.00275752 0.00351031 0.00389189 0.00349346 0.01606478]


In [16]:
# ExtraTrees Classifier

clf_ext = ExtraTreesClassifier(bootstrap=False, criterion='gini', 
                               max_features=0.1, min_samples_leaf=3, 
                               min_samples_split=2, n_estimators=100,
                               n_jobs=-1)

%time clf_ext.fit(X_train, y_train)
y_pred = clf_ext.predict(X_test)

print('fraction:', fraction)
print(clf_ext.score(X_test, y_test), '\n')
print(confusion_matrix(y_test, y_pred), '\n')
print(feature_importances(clf.ext))

Wall time: 18min 52s
fraction: 3
0.8260621419661132 

[[302315  27266   5885]
 [ 34091 146950  11451]
 [ 16326  19876  96392]] 

[0.01598173 0.00602292 0.00747779 0.00660891 0.00722444 0.00720368
 0.00723895 0.00723508 0.0209168  0.01326456 0.00693649 0.00601206
 0.0512395  0.03711365 0.05391975 0.02857365 0.04935964 0.05883397
 0.01486982 0.03025712 0.02119741 0.03835331 0.05296085 0.03073117
 0.04007977 0.02979854 0.03526601 0.01390352 0.01273685 0.01305409
 0.00879134 0.01136555 0.00417863 0.0085135  0.00888788 0.00280659
 0.00971187 0.01125185 0.01214509 0.01081509 0.0117858  0.01337232
 0.01367622 0.01130805 0.00864022 0.01399578 0.01274583 0.01048712
 0.00298907 0.00725779 0.00634211 0.00769236 0.00527811 0.00480141
 0.00474715 0.0024305  0.00288093 0.0033019  0.00287838 0.00309321
 0.00598014 0.00856064 0.00521275 0.00468997 0.0130128 ]


In [ ]:
# ExtraTrees Classifier

clf_ext = ExtraTreesClassifier(bootstrap=False, class_weight=None, 
                               criterion='gini', max_depth=None, 
                               max_features='auto', max_leaf_nodes=None,
                               min_impurity_decrease=0.0, 
                               min_impurity_split=None, min_samples_leaf=1, 
                               min_samples_split=2, 
                               min_weight_fraction_leaf=0.0, n_estimators=10, 
#                                n_jobs=None, 
                               n_jobs=-1, 
                               oob_score=False, 
                               random_state=None, verbose=0, warm_start=False)

%time clf_ext.fit(X_train, y_train)
y_pred = clf_ext.predict(X_test)

print('fraction:', fraction, 'score:', clf_ext.score(X_test, y_test), '\n')
print(confusion_matrix(y_test, y_pred), '\n')
print(feature_importances(clf_ext))

In [ ]:
# # ExtraTrees Classifier

# clf_ext = ExtraTreesClassifier(bootstrap=False, class_weight=None, 
#                                criterion='gini', max_depth=None, 
#                                max_features='auto', max_leaf_nodes=None,
#                                min_impurity_decrease=0.0, 
#                                min_impurity_split=None, min_samples_leaf=1, 
#                                min_samples_split=2, 
#                                min_weight_fraction_leaf=0.0, n_estimators=10, 
#                                n_jobs=None, oob_score=False, 
#                                random_state=None, verbose=0, warm_start=False)

# %time clf_ext.fit(X_train, y_train)
# y_pred = clf_ext.predict(X_test)

# print('fraction:', fraction, 'score:', clf_ext.score(X_test, y_test), '\n')
# print(confusion_matrix(y_test, y_pred), '\n')
# print(feature_importances(clf_ext))

In [ ]:
# # XGBClassifier
# param = {
#     'tree_method': 'gpu_hist', 
#     'gpu_id': 0
# }
# clf_xgb = xgb.XGBClassifier(**param)

# %time clf_xgb.fit(X_train, y_train)
# y_pred = clf_xgb.predict(X_test)

# print('fraction:', fraction, 'score:', clf_xgb.score(X_test, y_test), '\n')
# print(confusion_matrix(y_test, y_pred), '\n')
# print(feature_importances(clf_xgb))

In [ ]:
# # XGBClassifier
# # param = {
# #     'tree_method': 'gpu_hist', 
# #     'gpu_id': 0
# # }
# clf_xgb = xgb.XGBClassifier()

# %time clf_xgb.fit(X_train, y_train)
# y_pred = clf_xgb.predict(X_test)

# print('fraction:', fraction, 'score:', clf_xgb.score(X_test, y_test), '\n')
# print(confusion_matrix(y_test, y_pred), '\n')
# print(feature_importances(clf_xgb))

In [ ]:
# # XGBClassifier

# # clf_xgb = XGBClassifier()
# clf_xgb = xgb.XGBClassifier(base_score=0.5, booster='gbtree', 
#                             colsample_bylevel=1, colsample_bynode=1, 
#                             colsample_bytree=1, gamma=0, learning_rate=0.1, 
#                             max_delta_step=0, max_depth=3, min_child_weight=1,
#                             missing=None, n_estimators=100, n_jobs=1, 
# #                             nthread=None, 
#                             nthread=None, objective='multi:softprob', 
#                             random_state=0, reg_alpha=0, reg_lambda=1, 
#                             scale_pos_weight=1, seed=None, silent=None, 
#                             subsample=1, verbosity=1)

# %time clf_xgb.fit(X_train, y_train)
# y_pred = clf_xgb.predict(X_test)

# print('fraction:', fraction, 'score:', clf_xgb.score(X_test, y_test), '\n')
# print(confusion_matrix(y_test, y_pred), '\n')
# print(feature_importances(clf_xgb))

In [ ]:
# def load_DMatrix(Xt: 'np.array', yt: 'np.array', 
#                  frac: int) -> Tuple['np.array']:
#     step, i = Xt.size // frac, 0
    
#     while i <= frac:
#         st, ed = i * step, (i + 1) * step
        
# #         yield Xt[st: ed], yt[st: ed]
#         yield xgb.DMatrix(Xt[st: ed], label=yt[st: ed])
        
#         i += 1

In [ ]:
# for (x, y) in load_DMatrix(arr_X, arr_y, fraction):
#     print(x.size)
#     print(y.size)
# #     rret
# # 0
# # 1717435

In [ ]:
# # Specify sufficient boosting iterations to reach a minimum
# num_round = 3

# # Leave most parameters as default
# param = {
# #     'subsample': 1 / fraction,
#     'objective': 'multi:softmax',  # Specify multiclass classification
#     'num_class': 3,  # Number of possible output classes
#     'tree_method': 'gpu_hist'  # Use GPU accelerated algorithm
# }

# # Convert input data from numpy to XGBoost format
# dtrain = xgb.DMatrix(X_train, label=y_train, nthread=-1)
# dtest = xgb.DMatrix(X_test, label=y_test, nthread=-1)
# # evallist = [(dtrain, 'train'), (dtest, 'test')]
# evallist = [(dtest, 'test')]
# gpu_res = {}  # Store accuracy result

# # Train model
# tmp = time.time()
# bst = xgb.train(param, 
#                 dtrain, 
# #                 load_DMatrix(arr_X, arr_y, fraction),
#                 num_boost_round=num_round, 
#                 evals=evallist, evals_result=gpu_res)
# print(f'GPU Training Time: {time.time() - tmp} seconds')

# # path = r''
# # booster.save_binary(dmatrix_test_filename)

# # save model
# bst.save_model(os.path.join(model_folder, 'xgb_70.model'))
# # dump model
# bst.dump_model(os.path.join(model_folder, 'xgb_70.raw.txt'))

# # dump model with feature map
# # booster.dump_model('xgb_70.nice.txt', '../data/featmap.txt')
# # save dmatrix into binary buffer（数据集保存）
# # dtest.save_binary('dtest.buffer')

# # Repeat for CPU algorithm
# # tmp = time.time()
# # param['tree_method'] = 'hist'
# # cpu_res = {}
# # xgb.train(param, dtrain, num_round, evals=[(dtest, 'test')], evals_result=cpu_res)
# # print("CPU Training Time: %s seconds" % (str(time.time() - tmp)))

In [ ]:
# y_pred = bst.predict(dtest, ntree_limit=num_round)
# conf = confusion_matrix(y_test, y_pred)
# score = np.sum(conf.diagonal()) / np.sum(conf)
# print(conf, '\n')
# print('fraction:', fraction, 'score:', score, '\n')
# # print(feature_importances(booster))

In [ ]:
# md_pa = os.path.join(model_folder, r'model_no-settings.p')
# with gzip.GzipFile(md_pa, 'wb') as file:
#     pickle.dump(clf_xgb, file)

In [ ]:
# best_type = clf.fitted_pipeline_.steps[1]
# print(best_type, '\n')

# %time best_type.fit(X_train, y_train)
# # y_pred = best_type.predict(X_test)

# print(best_type.score(X_test, y_test), '\n')
# # print(confusion_matrix(y_test, y_pred), '\n')
# print(best_type.feature_importances_)

# md_pa = os.path.join(model_folder, r'model_clf2.dat')
# with open(md_pa, 'wb') as file:
#     pickle.dump(best_type, file)